# AMP
The [AMP Broker Hub](https://distributor.amp.com.au/policies) is a single page HTML.
1. Policy heading and text is inside divs of class amp-accordion. 
2. Spans with class ht-heading_3 are rewritten as H3. 
3. The only link to opther policy data is a PDF called the Security Property Location Guide, which is a 
a table of postcode.

The policy is wrtten to policy/AML/Broker_Hub/Home_loan_polices.html

In [1]:
import requests
from bs4 import BeautifulSoup
import shutil
import os

directory_path=r"./policy/AMP/Broker_Hub"
output_html = f"{directory_path}/policies.html"

if os.path.exists(directory_path):
    shutil.rmtree(directory_path)  

os.makedirs(directory_path) 

In [2]:
def create_empty_html_document():
    # Create a new BeautifulSoup object with a basic HTML structure
    soup = BeautifulSoup('', 'html.parser')
    
    # Create the html tag
    html_tag = soup.new_tag('html')
    soup.append(html_tag)
    
    # Create the head tag
    head_tag = soup.new_tag('head')
    html_tag.append(head_tag)
    
    # Add basic elements to the head
    # title_tag = soup.new_tag('title')
    # title_tag.string = 'Document Title'
    # head_tag.append(title_tag)
    
    # Add meta charset tag
    meta_tag = soup.new_tag('meta')
    meta_tag['charset'] = 'UTF-8'
    head_tag.append(meta_tag)
    
    # Create the body tag
    body_tag = soup.new_tag('body')
    html_tag.append(body_tag)
    
    # Return the formatted HTML string
    return soup

In [3]:

def extract_accordions(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Find all span elements with class "ht-heading_3" 
        ht_heading_spans = soup.find_all('span', class_='ht-heading_3')
        
        for span in ht_heading_spans:
            new_h3 = soup.new_tag('h3')
            new_h3.string = span.get_text()
            
            # Replace the parent paragraph with the new h3
            parent = span.find_parent('p')
            if parent:
                parent.replace_with(new_h3)
            else:
                # If there's no parent p tag, just replace the span itself
                span.replace_with(new_h3)

        output_soup = create_empty_html_document()
        body = output_soup.body
        accordions = soup.select('div.amp-accordion')
        for accordion in accordions:

            body.append(accordion)

        return str(output_soup)
            
    except requests.exceptions.RequestException as e:
        print(f"Error fetching the URL: {e}")
        return []


# Extract HTML inside div.ammp-accordions
html_text = extract_accordions("https://distributor.amp.com.au/policies")

with open(output_html, 'w', encoding='utf-8') as f:
    f.write(html_text)


In [ ]:
from IPython.display import HTML, display
HTML(html_text)